# Protein Ligand Complex MD Setup using BioExcel Building Blocks (biobb) and Gromacs

## Introduction

***
This notebook illustrate the process of **setting up a simulation system** containing a **protein (bcl-2, pdbcode 2xa0) in complex with a ligand (Genestein, Pubchem CID 5280961)**.

We are using different open source tools wrapped into Python **BioExcel Building Blocks library (biobb)**. 

For molecular dynamics we are using the Gromacs MD tools wrapped into the biobb library.

Based on the official Gromacs tutorial: http://www.mdtutorials.com/gmx/complex/index.html

***
**Biobb modules** used:

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to manipulate chemical data.
 - [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils):  Tools to modify or extract information from a PDB structure file.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [os](https://docs.python.org/3/library/os.html): Python miscellaneous operating system interfaces
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_protein-complex_md_setup.git
cd biobb_wf_protein-complex_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_Protein-Complex_MDsetup_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_protein-complex_md_setup/notebooks/biobb_Protein-Complex_MDsetup_tutorial.ipynb
  ``` 

***
### Pipeline steps:
 1. [Introduction](#intro)
 2. [Input Parameters](#input)
 3. [Extract Protein and Ligand Structures](#extract)
 4. [Fix Protein Structure](#fix)
 5. [Create Protein System Topology](#top)
 6. [Create ligand system topology](#ligtop)
 7. [Preparing Ligand Restraints](#restraints)
 8. [Create new protein-ligand complex structure file](#complex)
 9. [Create new protein-ligand complex topology file](#complextop)
 10. [Create Solvent Box](#box)
 11. [Fill the Box with Water Molecules](#water)
 12. [Adding Ions](#ions)
 13. [Energetically Minimize the System](#min)
 14. [Equilibrate the System (NVT)](#nvt)
 15. [Equilibrate the System (NPT)](#npt)
 16. [Free Molecular Dynamics Simulation](#free)
 17. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 18. [Output Files](#output)

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **dockedPdb**: PDB Complex resulting from docking process.
 - **pdbName**: PDB file of the protein-ligand complex structure resulting from docking process.
 - **ligandName**: Small molecule 3-letter code for the ligand structure (GEN for Genistein conformer).
 - **mol_charge**: Charge of the small molecule, needed to add hydrogen atoms.

In [2]:
dockedPdb  = "inputs/2xa0_A-Genistein.pdb"
pdbName    = "protein"
ligandName = "ligand"
ligandID   = "GEN"
mol_charge = 0

<a id="libraries"></a>
## Libraries and Functions

In [3]:
import nglview
import ipywidgets
import os
import zipfile

In [4]:
# Show protein-ligand complex
nglview.show_structure_file(dockedPdb)

NGLWidget()

<img src='images/img00.png' style='float: center;width:15%'></img>

<a id="extract"></a>
## Extracting Protein, Ligand and Protein-Ligand Complex

In [5]:
# Extracting Protein, Ligand and Protein-Ligand Complex to three different files
# Import module
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Create inputs/outputs
proteinFile = pdbName     +'.pdb'
ligandFile  = ligandName  +'.pdb'
complexFile = pdbName+'_' + ligandName + '.pdb'
print (">>> Files:", dockedPdb, ligandID, proteinFile, ligandFile, complexFile)

>>> Files: inputs/2xa0_A-Genistein.pdb GEN protein.pdb ligand.pdb protein_ligand.pdb


In [6]:
# Extract molecule
print ("\n>>> Extracting molecule...")
extract_molecule(input_structure_path=dockedPdb,
     output_molecule_path=proteinFile)

# Extract ligand 
print ("\n>>> Extracting ligand...")
prop = {
     'heteroatoms' : [{"name": ligandID}]
}

extract_heteroatoms(input_structure_path=dockedPdb,
     output_heteroatom_path=ligandFile, properties=prop)


# Create complex
print ("\n>>> Creating complex...")
cat_pdb(input_structure1=proteinFile, input_structure2=ligandFile,
       output_structure_path=complexFile)


>>> Extracting molecule...
2022-04-22 10:33:46,557 [MainThread  ] [INFO ]  Creating 7c67e2a7-be10-47c1-852f-55af11b17631 temporary folder
2022-04-22 10:33:47,094 [MainThread  ] [INFO ]  check_structure -i /home/lg/repos/javerianaMD/simulation/md-biobb/inputs/2xa0_A-Genistein.pdb -o protein.pdb --force_save --non_interactive command_list --list 7c67e2a7-be10-47c1-852f-55af11b17631/extract_prot.lst

2022-04-22 10:33:47,096 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:33:47,097 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure /home/lg/repos/javerianaMD/simulation/md-biobb/inputs/2xa0_A-Genistein.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 ( : Unknown (P:0.0% DNA:0.0% RNA:0.0% UNK:100.0%), A: Protein)

0

### Visualizing 3D structures 

In [7]:
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(proteinFile)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1
view2 = nglview.show_structure_file(ligandFile)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2
view3 = nglview.show_structure_file(complexFile)
view3.add_representation(repr_type='licorice', radius='.5', selection=ligandName)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3
ipywidgets.HBox([view1, view2, view3])

<img src='images/img01.png' style='float: center;width:50%'></img>

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.
***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**

Class to model the missing atoms in amino acid side chains of a PDB.
Model the missing atoms in amino acid side chains of a PDB using biobb_structure_checking if the use_modeller property is added the Modeller suite will also be used to rebuild the missing atoms.
***

In [8]:
# Check & Fix Protein Structure
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = pdbName +  '_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=proteinFile, 
               output_pdb_path=fixed_pdb)

2022-04-22 10:33:48,117 [MainThread  ] [INFO ]  check_structure -i protein.pdb -o protein_fixed.pdb --force_save fixside --fix ALL

2022-04-22 10:33:48,119 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:33:48,120 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure protein.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  137
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1145

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. r

0

In [9]:
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(fixed_pdb, default_representation=False)
view1.add_licorice(
    "sidechainAttached "
)  # color="#FF0000" or color=0xFF0000 would also work!
view1.center()
view1

NGLWidget()

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
- Force field used is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): 
    * AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn).<br>
    
- Water molecules type used is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
- Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
    
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_md.gromacs.pdb2gmx**

The GROMACS pdb2gmx module, reads a .pdb (or .gro) file, reads some database files, adds hydrogens to the molecules and generates coordinates in GROMACS (GROMOS), or optionally .pdb, format and a topology in GROMACS format. These files can subsequently be processed to generate a run input file.

***

In [10]:
# Create Protein system topology
# Import module
from biobb_md.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro     = pdbName + '_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbName + '_pdb2gmx_top.zip'
prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb,
        output_gro_path=output_pdb2gmx_gro,
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=prop)

2022-04-22 10:33:48,273 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2022-04-22 10:33:48,275 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:33:49,115 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f protein_fixed.pdb -o protein_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2022-04-22 10:33:49,116 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:33:49,117 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading protein_fixed.pdb...
Read '', 1145 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 137 residues with 1145 atoms

  chain  #res #atoms
  1 'A'   137   1145  

Reading residue database... (Amber99sb-ildn)
Processing chain 1 'A' (1145 atoms, 137 resi

0

<a id="ligtop"></a>
***
## Create ligand system topology
- **Building GROMACS topology** corresponding to the ligand structure.<br>
- Force field used is **amberGAFF**: [General AMBER Force Field](http://ambermd.org/antechamber/gaff.html), designed for rational drug design.<br>
- [Step 1](#ligandTopologyStep1): Add **hydrogen atoms** if missing.
- [Step 2](#ligandTopologyStep2): **Energetically minimize the system** with the new hydrogen atoms. 
- [Step 3](#ligandTopologyStep3): Generate **ligand topology** (parameters). 

***
**Building Blocks** used:
 - [ReduceAddHydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_add_hydrogens) from **biobb_chemistry.ambertools.reduce_add_hydrogens**: 
     * Module for adding hydrogens to a given structure.
 
 
 - [BabelMinimize](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_minimize) from **biobb_chemistry.babelm.babel_minimize**:
     * Energetically minimizes small molecules. 
     
 
 - [AcpypeParamsGMX](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_gmx) from **biobb_chemistry.acpype.acpype_params_gmx**:
     * Acpype is a tool based in Python to use Antechamber to generate topologies for chemical compounds and to interface with others python applications like CCPN or ARIA
***

<a id="ligandTopologyStep1"></a>
### Step 1: Add **hydrogen atoms**

In [11]:
# Create Ligand system topology, STEP 1
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandName + '_reduce_H.pdb' 
prop = {
    'nuclear' : 'true'
}

# Create and launch bb
reduce_add_hydrogens(input_path=ligandFile,
                   output_path=output_reduce_h,
                   properties=prop)


2022-04-22 10:33:49,171 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:33:49,322 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT ligand.pdb > ligand_reduce_H.pdb

2022-04-22 10:33:49,323 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:33:49,325 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "ligand.pdb"
Database of HETATM connections: "/opt/miniconda3/envs/biobb_env//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
Found 0 hydrogens (0 hets)
Standardized 0 hydrogens (0 hets)
Added 0 hydrogens (0 hets)
Removed 0 hydrogens (0 hets)
If you publish work which uses redu

0

<a id="ligandTopologyStep2"></a>
### Step 2: **Energetically minimize the system** with the new hydrogen atoms. 

In [12]:
# Create Ligand system topology, STEP 2
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandName + '.H.min.mol2'                              
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}

# Create and launch bb
babel_minimize(input_path = output_reduce_h,
              output_path = output_babel_min,
              properties  = prop)

2022-04-22 10:33:49,339 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2022-04-22 10:33:49,340 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2022-04-22 10:33:49,341 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2022-04-22 10:33:49,342 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2022-04-22 10:33:49,344 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2022-04-22 10:33:49,345 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2022-04-22 10:33:49,345 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:33:49,947 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb ligand_reduce_H.pdb -omol2 > ligand.H.min.mol2

2022-04-22 10:33:49,948 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:33:49,949 [MainThread  ] [INFO ]  
A T O M   T Y P E S

IDX	TYPE	RING
1	os	AR
2	o	NO
3	cd	AR
4	cd	AR
5	cd	AR
6	

0

### Visualizing 3D structures
Visualizing the small molecule generated **PDB structures** using **NGL**:  
- **Original Ligand Structure** (Left)
- **Ligand Structure with hydrogen atoms added** (with Reduce program) (Center)
- **Ligand Structure with hydrogen atoms added** (with Reduce program), **energy minimized** (with Open Babel) (Right) 

In [13]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<img src='images/ngl4-ngl5-ngl6.png' style='float: left;width:70%'></img>

<a id="ligandTopologyStep3"></a>
### Step 3: Generate **ligand topology** (parameters).
We use here ACPYPE tool (http://www.biomedcentral.com/1756-0500/5/367), a wrapper script around the ANTECHAMBER software (http://ambermd.org/) to generate topologies for chemical compounds. Topologies files to be generated so far: CNS/XPLOR, GROMACS, CHARMM and AMBER. Topologies generated by acpype/Antechamber are based on General Amber Force Field (GAFF) and should be used only with compatible force fields like AMBER and its variants.

In [14]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

# Create prop dict and inputs/outputs
output_acpype_gro = ligandName + '_params.gro'
output_acpype_itp = ligandName + '_params.itp'
output_acpype_top = ligandName + '_params.top'
output_acpype     = ligandName + '_params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_gmx(input_path=output_babel_min, 
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop);

2022-04-22 10:33:50,333 [MainThread  ] [INFO ]  Running acpype, this execution can take a while
2022-04-22 10:33:50,334 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:11,568 [MainThread  ] [INFO ]  acpype -i /home/lg/repos/javerianaMD/simulation/md-biobb/ligand.H.min.mol2 -b ligand_params.aKbnj2 -n 0

2022-04-22 10:34:11,569 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:11,569 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2022 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'GEN'....
Checking parameters for unit 'GEN'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote++++++++++++++++++++++++++++++++++++++++++++++

<a id="restraints"></a>
***
## Preparing Ligand Restraints
- In subsequent steps of the pipeline, such as the equilibration stages of the **protein-ligand complex** system, it is recommended to apply some **restraints** to the small molecule, to avoid a possible change in position due to protein repulsion. 

- **Position restraints** will be applied to the ligand, using a **force constant of 1000 KJ/mol\*nm^2** on the three coordinates: x, y and z. In this steps the **restriction files** will be created and integrated in the **ligand topology**.
- [Step 1](#restraintsStep1): Creating an index file with a new group including just the **small molecule heavy atoms**.
- [Step 2](#restraintsStep2): Generating the **position restraints** file.
***
**Building Blocks** used:
 - [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_md.gromacs.make_ndx**:
     * The GROMACS make_ndx module, generates an index file using the atoms of the selection.
     
     
 - [Genrestr](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genrestr) from **biobb_md.gromacs.genrestr**:
     * Wrapper of the GROMACS genrestr module. The GROMACS genrestr module, produces an #include file for a topology containing a list of atom numbers and three force constants for the x-, y-, and z-direction based on the contents of the -f file. A single isotropic force constant may be given on the command line instead of three components.
***

<a id="restraintsStep1"></a>
### Step 1: Creating an index file for the small molecule heavy atoms

In [15]:
# MakeNdx: Creating index file with a new group (small molecule heavy atoms)
from biobb_md.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_ligand_ndx = ligandName + '_index.ndx'
prop = {
    'selection': "0 & ! a H*"
}

# Create and launch bb
make_ndx(input_structure_path=output_acpype_gro,
        output_ndx_path=output_ligand_ndx,
        properties=prop)

2022-04-22 10:34:11,614 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2022-04-22 10:34:11,616 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:11,643 [MainThread  ] [INFO ]  echo -e '0 & ! a H*\nq' | gmx -nobackup -nocopyright make_ndx -f ligand_params.gro -o ligand_index.ndx

2022-04-22 10:34:11,646 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:11,649 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :    20 atoms
  1 Other               :    20 atoms
  2 GEN                 :    20 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' 

0

<a id="restraintsStep2"></a>
### Step 2: Generating the position restraints file

In [16]:
# Genrestr: Generating the position restraints file
from biobb_md.gromacs.genrestr import genrestr

# Create prop dict and inputs/outputs
output_restraints_top = ligandName + '_posres.itp'
prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "0"    # Modified from "System" to "0"
}

# Create and launch bb
genrestr(input_structure_path=output_acpype_gro,
         input_ndx_path=output_ligand_ndx,
         output_itp_path=output_restraints_top,
         properties=prop)

2022-04-22 10:34:11,717 [MainThread  ] [INFO ]  GROMACS Genrestr 20191 version detected
2022-04-22 10:34:11,719 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:11,754 [MainThread  ] [INFO ]  echo "0" | gmx -nobackup -nocopyright genrestr -f ligand_params.gro -o ligand_posres.itp -fc 1000 1000 1000 -n ligand_index.ndx

2022-04-22 10:34:11,758 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:11,759 [MainThread  ] [INFO ]  Select group to position restrain
Selected 0: 'System'

2022-04-22 10:34:11,765 [MainThread  ] [INFO ]                       :-) GROMACS - gmx genrestr, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/simulation/md-biobb
Command line:
  gmx -nobackup -nocopyright genrestr -f ligand_params.gro -o ligand_posres.itp -fc 1000 1000 1000 -n ligand_index.ndx


Reading structure file
Group     0 (         System) has    20 elements
Group     1 (          

0

<a id="complex"></a>
***
## Create new protein-ligand complex structure file
Building new **protein-ligand complex** PDB file with:
- The new **protein system** with fixed problems from *Fix Protein Structure* step and hydrogens atoms added from *Create Protein System Topology* step.
- The new **ligand system** with hydrogens atoms added from *Create Ligand System Topology* step. 

This new structure is needed for **GROMACS** as it is **force field-compliant**, it **has all the new hydrogen atoms**, and the **atom names are matching the newly generated protein and ligand topologies**.
***
**Building Blocks** used:
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**:
     * Wrapper of the GROMACS trjconv module for converting between GROMACS compatible structure file formats and/or extracting a selection of atoms. GROMACS trjconv module can convert trajectory files in many ways
***

In [17]:
print (pdbName)
print (ligandName)
print (output_pdb2gmx_gro)
print (output_acpype_gro)

protein
ligand
protein_pdb2gmx.gro
ligand_params.gro


### Protein: Convert gro (with hydrogens) to pdb

In [18]:
# biobb analysis module
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
proteinFile_H = pdbName + '_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path = output_pdb2gmx_gro,
                input_top_path       = output_pdb2gmx_gro,
                output_str_path      = proteinFile_H, 
                properties           = prop)

2022-04-22 10:34:11,802 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:11,839 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_pdb2gmx.gro -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_pdb2gmx.gro -o protein_complex_H.pdb

2022-04-22 10:34:11,841 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:11,845 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2022-04-22 10:34:11,846 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent

0

### Ligand: Convert gro (with hydrogens) to pdb

In [19]:
# Convert gro (with hydrogens) to pdb (LIGAND)
ligandFile_H = ligandName + '_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path = output_acpype_gro,
              input_top_path         = output_acpype_gro,
              output_str_path        = ligandFile_H, 
              properties             = prop)

2022-04-22 10:34:11,863 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:11,883 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /home/lg/repos/javerianaMD/simulation/md-biobb/ligand_params.gro -s /home/lg/repos/javerianaMD/simulation/md-biobb/ligand_params.gro -o ligand_complex_H.pdb

2022-04-22 10:34:11,885 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:11,887 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2022-04-22 10:34:11,891 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hind

0

### Concatenating both PDB files: Protein + Ligand

In [20]:
# Concatenating both PDB files: Protein + Ligand
complexFile_H = pdbName + '_' + ligandName + '_complex_H.pdb'

# Create and launch bb
cat_pdb(input_structure1      = proteinFile_H,
        input_structure2      = ligandFile_H,
        output_structure_path = complexFile_H);

2022-04-22 10:34:11,918 [MainThread  ] [INFO ]  File protein_ligand_complex_H.pdb created


<a id="complextop"></a>
***
## Create new protein-ligand complex topology file
Building new **protein-ligand complex** GROMACS topology file with:
- The new **protein system** topology generated from *Create Protein System Topology* step.
- The new **ligand system** topology generated from *Create Ligand System Topology* step. 

NOTE: From this point on, the **protein-ligand complex structure and topology** generated can be used in a regular MD setup.
***
**Building Blocks** used:
 - [AppendLigand](https://biobb-md.readthedocs.io/en/latest/modules.html) from **biobb_md.gromacs_extra.append_ligand**  (NOTE: link should be updated with the documentation):
     * This class takes a ligand ITP file and inserts it in a topology.
This module automatizes the process of inserting a ligand ITP file in a GROMACS topology.
***

In [21]:
# AppendLigand: Append a ligand to a GROMACS topology
# Import module
from biobb_md.gromacs_extra.append_ligand import append_ligand

# Create prop dict and inputs/outputs
output_complex_top = pdbName + '_' + ligandName + '_complex.top.zip'

posresifdef = "POSRES_" + ligandName.upper()
prop = {
    'posres_name': posresifdef
}

# Create and launch bb
append_ligand(input_top_zip_path   = output_pdb2gmx_top_zip,
             input_posres_itp_path = output_restraints_top,
             input_itp_path        = output_acpype_itp, 
             output_top_zip_path   = output_complex_top,
             properties            = prop)

2022-04-22 10:34:11,941 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_pdb2gmx_top.zip
2022-04-22 10:34:11,943 [MainThread  ] [INFO ]  to:
2022-04-22 10:34:11,944 [MainThread  ] [INFO ]  ['1a8b7906-7cf0-4af4-a781-6f5f7f13f998/p2g.top', '1a8b7906-7cf0-4af4-a781-6f5f7f13f998/posre.itp']
2022-04-22 10:34:11,945 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:34:11,946 [MainThread  ] [INFO ]  protein_pdb2gmx_top.zip
2022-04-22 10:34:11,951 [MainThread  ] [INFO ]  To: 
2022-04-22 10:34:11,952 [MainThread  ] [INFO ]  1a8b7906-7cf0-4af4-a781-6f5f7f13f998/p2g.top
2022-04-22 10:34:11,953 [MainThread  ] [INFO ]  1a8b7906-7cf0-4af4-a781-6f5f7f13f998/posre.itp
2022-04-22 10:34:11,999 [MainThread  ] [INFO ]  Compressing topology to: protein_ligand_complex.top.zip
2022-04-22 10:34:12,001 [MainThread  ] [INFO ]  Ignored file 1a8b7906-7cf0-4af4-a781-6f5f7f13f998/amber99sb-ildn.ff/forcefield.itp
2022-04-22 10:34:12,022 [MainThread  ] [INFO ]  Ignored file 1

0

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein-ligand complex** to fill it with water molecules.<br>
**Truncated octahedron** box is used for the unit cell. This box type is the one which best reflects the geometry of the solute/protein, in this case a **globular protein**, as it approximates a sphere. It is also convenient for the computational point of view, as it accumulates **less water molecules at the corners**, reducing the final number of water molecules in the system and making the simulation run faster.<br> A **protein to box** distance of **0.8 nm** is used, and the protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_md.gromacs.editconf**:
     * The GROMACS solvate module generates a box around the selected structure.
***

In [22]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbName + '_'+ ligandName + '_complex_editconf.gro'

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 0.8
}

# Create and launch bb
editconf(input_gro_path=complexFile_H, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2022-04-22 10:34:12,071 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2022-04-22 10:34:12,072 [MainThread  ] [INFO ]  Centering molecule in the box.
2022-04-22 10:34:12,074 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.80
2022-04-22 10:34:12,075 [MainThread  ] [INFO ]  Box type: octahedron
2022-04-22 10:34:12,076 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:34:12,120 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f protein_ligand_complex_H.pdb -o protein_ligand_complex_editconf.gro -d 0.8 -bt octahedron -c

2022-04-22 10:34:12,122 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:34:12,124 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 2252 atoms
Volume: 2534.49 nm^3, corresponds to roughly 1140500 electrons
No velocities found
    system size :  4.104  3.681  3.525 (nm)
    diameter    :  4.778               (nm)
    center      :  3.365 -1.411 -1

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein-ligand complex** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_md.gromacs.solvate**:
     * The GROMACS solvate module, generates a box of solvent around the selected structure.
***

In [23]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro     = pdbName + '_' + ligandName + '_solvate.gro'
output_solvate_top_zip = pdbName + '_' + ligandName + '_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro,
        output_gro_path=output_solvate_gro,
        input_top_zip_path=output_complex_top,
        output_top_zip_path=output_solvate_top_zip)

2022-04-22 10:34:12,179 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2022-04-22 10:34:12,188 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_complex.top.zip
2022-04-22 10:34:12,189 [MainThread  ] [INFO ]  to:
2022-04-22 10:34:12,189 [MainThread  ] [INFO ]  ['d53f7d18-be23-40f0-a0a8-f880691cfa56/ligand.top', 'd53f7d18-be23-40f0-a0a8-f880691cfa56/ligand_params.itp', 'd53f7d18-be23-40f0-a0a8-f880691cfa56/ligand_posres.itp', 'd53f7d18-be23-40f0-a0a8-f880691cfa56/posre.itp']
2022-04-22 10:34:12,190 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:34:12,191 [MainThread  ] [INFO ]  protein_ligand_complex.top.zip
2022-04-22 10:34:12,191 [MainThread  ] [INFO ]  To: 
2022-04-22 10:34:12,192 [MainThread  ] [INFO ]  d53f7d18-be23-40f0-a0a8-f880691cfa56/ligand.top
2022-04-22 10:34:12,193 [MainThread  ] [INFO ]  d53f7d18-be23-40f0-a0a8-f880691cfa56/ligand_params.itp
2022-04-22 10:34:12,194 [MainThread  ] [INFO ]  d53f7d18-be23-40f0

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **solvent box** using **NGL**<br>
Note the **octahedral box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the box.

In [24]:
#Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandName)
view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='images/ngl7.png' width="50%" height="50%"></img>

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein-ligand complex** and reach a desired ionic concentration.
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system and reach a **0.05 molar ionic concentration**
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
     * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR.
     
     
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_md.gromacs.genion**:
     * The GROMACS genion module randomly replaces solvent molecules with monoatomic ions. The group of solvent molecules should be continuous and all molecules should have the same number of atoms.
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation

In [25]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'minimization',
    'maxwarn': 1
}
output_gppion_tpr = pdbName + '_' + ligandName + '_complex_gppion.tpr'

# Create and launch bb
grompp(input_gro_path=output_solvate_gro,
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,
       properties=prop);

2022-04-22 10:34:13,016 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-22 10:34:13,022 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_solvate_top.zip
2022-04-22 10:34:13,023 [MainThread  ] [INFO ]  to:
2022-04-22 10:34:13,024 [MainThread  ] [INFO ]  ['2c79ed34-df5e-4296-81b4-9169dff81815/ligand.top', '2c79ed34-df5e-4296-81b4-9169dff81815/ligand_params.itp', '2c79ed34-df5e-4296-81b4-9169dff81815/ligand_posres.itp', '2c79ed34-df5e-4296-81b4-9169dff81815/posre.itp']
2022-04-22 10:34:13,025 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:34:13,026 [MainThread  ] [INFO ]  protein_ligand_solvate_top.zip
2022-04-22 10:34:13,027 [MainThread  ] [INFO ]  To: 
2022-04-22 10:34:13,028 [MainThread  ] [INFO ]  2c79ed34-df5e-4296-81b4-9169dff81815/ligand.top
2022-04-22 10:34:13,028 [MainThread  ] [INFO ]  2c79ed34-df5e-4296-81b4-9169dff81815/ligand_params.itp
2022-04-22 10:34:13,029 [MainThread  ] [INFO ]  2c79ed34-df5e-4296-

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system and reach a 0.05 molar concentration
Replace **solvent molecules** with **ions** to **neutralize** the system and reaching a **0.05 molar ionic concentration**

In [26]:
# Genion: Adding ions to reach a 0.05 molar concentration
from biobb_md.gromacs.genion import genion

# Create prop dict and inputs/outputs
prop={
    'neutral':True,
    'concentration':0.05
}
output_genion_gro = pdbName + '_' + ligandName+'_genion.gro'
output_genion_top_zip = pdbName + '_' + ligandName+'_genion_top.zip'

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr,
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip,
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2022-04-22 10:34:13,625 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2022-04-22 10:34:13,633 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_solvate_top.zip
2022-04-22 10:34:13,636 [MainThread  ] [INFO ]  to:
2022-04-22 10:34:13,637 [MainThread  ] [INFO ]  ['3797d416-513a-4c8d-9b71-f38a9d9f9f26/ligand.top', '3797d416-513a-4c8d-9b71-f38a9d9f9f26/ligand_params.itp', '3797d416-513a-4c8d-9b71-f38a9d9f9f26/ligand_posres.itp', '3797d416-513a-4c8d-9b71-f38a9d9f9f26/posre.itp']
2022-04-22 10:34:13,638 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:34:13,646 [MainThread  ] [INFO ]  protein_ligand_solvate_top.zip
2022-04-22 10:34:13,648 [MainThread  ] [INFO ]  To: 
2022-04-22 10:34:13,651 [MainThread  ] [INFO ]  3797d416-513a-4c8d-9b71-f38a9d9f9f26/ligand.top
2022-04-22 10:34:13,652 [MainThread  ] [INFO ]  3797d416-513a-4c8d-9b71-f38a9d9f9f26/ligand_params.itp
2022-04-22 10:34:13,653 [MainThread  ] [INFO ]  3797d416-513a-4c8d-

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **ionic concentration** using **NGL**

In [27]:
#Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandName)

view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')

view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','400px'])
view.camera='orthographic'
view.center()
view

NGLWidget()

<img src='images/ngl8.png' width="50%" height="50%"></img>

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein-ligand complex** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **protein-ligand complex** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
     * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR.
     
     
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun**:
     * MDRun is the main computational chemistry engine within GROMACS. It performs Molecular Dynamics simulations, but it can also perform Stochastic Dynamics, Energy Minimization, test particle insertion or (re)calculation of energies.
     
     
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy**:
     * GROMACS energy extracts energy components from an energy file.
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
Method used to run the **energy minimization** is a **steepest descent**, with a **maximum force of 500 KJ/mol\*nm^2**, and a minimization **step size of 1fs**. The **maximum number of steps** to perform if the maximum force is not reached is **5,000 steps**. 

In [28]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000',
        'emstep': 0.01,
        'emtol':'500'
    },
    'simulation_type':'minimization'
}
output_gppmin_tpr = pdbName + '_' + ligandName + '_gppmin.tpr'

# Create and launch bb
grompp(input_gro_path     = output_genion_gro,
       input_top_zip_path = output_genion_top_zip,
       output_tpr_path    = output_gppmin_tpr,
       properties         = prop)

2022-04-22 10:34:14,322 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-22 10:34:14,326 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_genion_top.zip
2022-04-22 10:34:14,327 [MainThread  ] [INFO ]  to:
2022-04-22 10:34:14,327 [MainThread  ] [INFO ]  ['a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/ligand.top', 'a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/ligand_params.itp', 'a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/ligand_posres.itp', 'a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/posre.itp']
2022-04-22 10:34:14,328 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:34:14,329 [MainThread  ] [INFO ]  protein_ligand_genion_top.zip
2022-04-22 10:34:14,329 [MainThread  ] [INFO ]  To: 
2022-04-22 10:34:14,330 [MainThread  ] [INFO ]  a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/ligand.top
2022-04-22 10:34:14,331 [MainThread  ] [INFO ]  a4b947b3-ccea-46a9-8f68-8aa6327b8a6e/ligand_params.itp
2022-04-22 10:34:14,332 [MainThread  ] [INFO ]  a4b947b3-ccea-46a9-8f

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step.

In [29]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = pdbName+'_'+ligandName+'_min.trr'
output_min_gro = pdbName+'_'+ligandName+'_min.gro'
output_min_edr = pdbName+'_'+ligandName+'_min.edr'
output_min_log = pdbName+'_'+ligandName+'_min.log'

# Create and launch bb
mdrun(input_tpr_path  = output_gppmin_tpr,
      output_trr_path = output_min_trr, 
      output_gro_path = output_min_gro,
      output_edr_path = output_min_edr, 
      output_log_path = output_min_log)

2022-04-22 10:34:14,852 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-22 10:34:14,853 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:35:25,417 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppmin.tpr -o protein_ligand_min.trr -c protein_ligand_min.gro -e protein_ligand_min.edr -g protein_ligand_min.log

2022-04-22 10:35:25,418 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:35:25,419 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/simulation/md-biobb
Command line:
  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppmin.tpr -o protein_ligand_min.trr -c protein_ligand_min.gro -e protein_ligand_min.edr -g protein_ligand_min.log

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings mor

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [30]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbName + '_' + ligandName + '_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path = output_min_edr, 
          output_xvg_path    = output_min_ene_xvg, 
          properties         = prop)

2022-04-22 10:35:25,440 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:35:25,461 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_min.edr -o protein_ligand_min_ene.xvg -xvg none < 32cc51a2-53ea-4a3b-8945-2fd2ffdf7def/instructions.in

2022-04-22 10:35:25,469 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:35:25,470 [MainThread  ] [INFO ]  
Statistics over 1454 steps [ 0.0000 through 1453.0000 ps ], 1 data sets
All statistics are over 1150 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -292857      18000    75481.1    -114387  (kJ/mol)

2022-04-22 10:35:25,471 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    

0

In [31]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig1.png' width="50%" height="50%"></img>

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein-ligand complex** system in NVT ensemble (constant Number of particles, Volume and Temperature). To avoid temperature coupling problems, a new *"system"* group will be created including the **protein** + the **ligand** to be assigned to a single thermostatting group.

- [Step 1](#eqNVTStep1): Creating an index file with a new group including the **protein-ligand complex**.
- [Step 2](#eqNVTStep3): Creating portable binary run file for system equilibration
- [Step 3](#eqNVTStep3): Equilibrate the **protein-ligand complex** with NVT ensemble.
- [Step 4](#eqNVTStep4): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_md.gromacs.make_ndx**:
    * The GROMACS make_ndx module, generates an index file using the atoms of the selection.
    

- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
    * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR. 
    
    
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun**:
    * MDRun is the main computational chemistry engine within GROMACS. It performs Molecular Dynamics simulations, but it can also perform Stochastic Dynamics, Energy Minimization, test particle insertion or (re)calculation of energies.
    
    
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy**:
    * GROMACS energy extracts energy components from an energy file. 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating an index file with a new group including the **protein-ligand complex**

In [32]:
# MakeNdx: Creating index file with a new group (protein-ligand complex)
from biobb_md.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_complex_ndx = pdbName + '_' + ligandName + '_index.ndx'
prop = {
    'selection': "\"Protein\"|\"Other\"" 
}

# Create and launch bb
make_ndx(input_structure_path = output_min_gro,
        output_ndx_path       = output_complex_ndx,
        properties=prop)

2022-04-22 10:35:26,023 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2022-04-22 10:35:26,024 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:35:26,137 [MainThread  ] [INFO ]  echo -e '"Protein"|"Other"\nq' | gmx -nobackup -nocopyright make_ndx -f protein_ligand_min.gro -o protein_ligand_index.ndx

2022-04-22 10:35:26,139 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:35:26,140 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:   137    Protein residues
There are:     1      Other residues
There are:  5792      Water residues
There are:    16        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 19644 atoms
  1 Protein             :  2232 atoms
  2 Protein-H           :  1145 atoms
  3 C-alpha             :   137 atoms
  4

0

<a id="eqNVTStep2"></a>
### Step 2: Creating portable binary run file for system equilibration (NVT)
Note that for the purposes of temperature coupling, the **protein-ligand complex** (*Protein_Other*) is considered as a single entity.

In [33]:
# Grompp: Creating portable binary run file for NVT System Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbName+'_'+ligandName+'gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'nvt'
}

# Create and launch bb
grompp(input_gro_path     = output_min_gro,
       input_top_zip_path = output_genion_top_zip,
       input_ndx_path     = output_complex_ndx,
       output_tpr_path    = output_gppnvt_tpr,
       properties         = prop)

2022-04-22 10:35:26,203 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-22 10:35:26,209 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_genion_top.zip
2022-04-22 10:35:26,211 [MainThread  ] [INFO ]  to:
2022-04-22 10:35:26,211 [MainThread  ] [INFO ]  ['09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/ligand.top', '09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/ligand_params.itp', '09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/ligand_posres.itp', '09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/posre.itp']
2022-04-22 10:35:26,216 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:35:26,217 [MainThread  ] [INFO ]  protein_ligand_genion_top.zip
2022-04-22 10:35:26,218 [MainThread  ] [INFO ]  To: 
2022-04-22 10:35:26,219 [MainThread  ] [INFO ]  09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/ligand.top
2022-04-22 10:35:26,221 [MainThread  ] [INFO ]  09ebb7d8-19d7-4c2d-8c84-f20a10692ed8/ligand_params.itp
2022-04-22 10:35:26,221 [MainThread  ] [INFO ]  09ebb7d8-19d7-4c2d-8c

0

<a id="eqNVTStep3"></a>
### Step 3: Running NVT equilibration

In [34]:
# Mdrun: Running NVT System Equilibration 
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = pdbName+'_'+ligandName+'_nvt.trr'
output_nvt_gro = pdbName+'_'+ligandName+'_nvt.gro'
output_nvt_edr = pdbName+'_'+ligandName+'_nvt.edr'
output_nvt_log = pdbName+'_'+ligandName+'_nvt.log'
output_nvt_cpt = pdbName+'_'+ligandName+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr,
      output_trr_path=output_nvt_trr,
      output_gro_path=output_nvt_gro,
      output_edr_path=output_nvt_edr,
      output_log_path=output_nvt_log,
      output_cpt_path=output_nvt_cpt)

2022-04-22 10:35:26,957 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-22 10:35:26,959 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:37:11,856 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s protein_ligandgppnvt.tpr -o protein_ligand_nvt.trr -c protein_ligand_nvt.gro -e protein_ligand_nvt.edr -g protein_ligand_nvt.log -cpo protein_ligand_nvt.cpt

2022-04-22 10:37:11,857 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:37:11,858 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/simulation/md-biobb
Command line:
  gmx -nobackup -nocopyright mdrun -s protein_ligandgppnvt.tpr -o protein_ligand_nvt.trr -c protein_ligand_nvt.gro -e protein_ligand_nvt.edr -g protein_ligand_nvt.log -cpo protein_ligand_nvt.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be bet

0

<a id="eqNVTStep4"></a>
### Step 4: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [35]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbName + '_' + ligandName + '_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path = output_nvt_edr, 
          output_xvg_path    =output_nvt_temp_xvg, 
          properties=prop)

2022-04-22 10:37:11,872 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:37:11,888 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_nvt.edr -o protein_ligand_nvt_temp.xvg -xvg none < f2178399-1c4b-4e14-a24a-e5d6cb49a5d9/instructions.in

2022-04-22 10:37:11,889 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:37:11,893 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 1 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 298.671        1.2     7.0413    7.43239  (K)

2022-04-22 10:37:11,894 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar     

0

In [36]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig2.png' width="50%" height="50%"></img>

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein-ligand complex** system in NPT ensemble (constant Number of particles, Pressure and Temperature) .
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein-ligand complex** with NPT ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

In [37]:
# Grompp: Creating portable binary run file for (NPT) System Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbName+'_'+ligandName+'_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnpt_tpr,
       input_cpt_path=output_nvt_cpt,
       properties=prop)

2022-04-22 10:37:11,994 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-22 10:37:12,001 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_genion_top.zip
2022-04-22 10:37:12,002 [MainThread  ] [INFO ]  to:
2022-04-22 10:37:12,004 [MainThread  ] [INFO ]  ['58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/ligand.top', '58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/ligand_params.itp', '58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/ligand_posres.itp', '58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/posre.itp']
2022-04-22 10:37:12,005 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:37:12,007 [MainThread  ] [INFO ]  protein_ligand_genion_top.zip
2022-04-22 10:37:12,008 [MainThread  ] [INFO ]  To: 
2022-04-22 10:37:12,008 [MainThread  ] [INFO ]  58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/ligand.top
2022-04-22 10:37:12,009 [MainThread  ] [INFO ]  58f16044-d8bc-4589-ad76-e7e0a1b5d6f2/ligand_params.itp
2022-04-22 10:37:12,011 [MainThread  ] [INFO ]  58f16044-d8bc-4589-ad

0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [38]:
# Mdrun: Running NPT System Equilibration
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = pdbName+'_'+ligandName+'_npt.trr'
output_npt_gro = pdbName+'_'+ligandName+'_npt.gro'
output_npt_edr = pdbName+'_'+ligandName+'_npt.edr'
output_npt_log = pdbName+'_'+ligandName+'_npt.log'
output_npt_cpt = pdbName+'_'+ligandName+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr,
      output_trr_path=output_npt_trr,
      output_gro_path=output_npt_gro,
      output_edr_path=output_npt_edr,
      output_log_path=output_npt_log,
      output_cpt_path=output_npt_cpt)

2022-04-22 10:37:12,541 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-22 10:37:12,542 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:39:01,763 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppnpt.tpr -o protein_ligand_npt.trr -c protein_ligand_npt.gro -e protein_ligand_npt.edr -g protein_ligand_npt.log -cpo protein_ligand_npt.cpt

2022-04-22 10:39:01,764 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:39:01,765 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/simulation/md-biobb
Command line:
  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppnpt.tpr -o protein_ligand_npt.trr -c protein_ligand_npt.gro -e protein_ligand_npt.edr -g protein_ligand_npt.log -cpo protein_ligand_npt.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be b

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [39]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbName+'_'+ligandName+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2022-04-22 10:39:01,782 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:39:01,804 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_npt.edr -o protein_ligand_npt_PD.xvg -xvg none < 5d1c8b1e-703d-4595-bf75-a52e6cec60c6/instructions.in

2022-04-22 10:39:01,805 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:39:01,806 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 2 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                    32.4553         19    244.506   -41.7307  (bar)
Density                     1038.43          2    6.44433    12.6402  (kg/m^3)

2022-04-22 10:39:01,807 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      R

0

In [40]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<img src='images/fig3.png' width="50%" height="50%"></img>

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein-ligand complex** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein-ligand complex**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms**:
     * Wrapper of the GROMACS rms module for performing a Root Mean Square deviation (RMSd) analysis from a given GROMACS compatible trajectory.
     
     
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr**:
     * Wrapper of the GROMACS gyrate module for computing the radius of gyration (Rgyr) of a molecule about the x-, y- and z-axes, as a function of time, from a given GROMACS compatible trajectory.
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

In [41]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        #'nsteps':'500000' # 1 ns (500,000 steps x 2fs per step)
        #'nsteps':'5000' # 10 ps (5,000 steps x 2fs per step)
        'nsteps':'25000' # 50 ps (25,000 steps x 2fs per step)
    },
    'simulation_type':'free'
}
output_gppmd_tpr = pdbName+'_'+ligandName + '_gppmd.tpr'

# Create and launch bb
grompp(input_gro_path     = output_npt_gro,
       input_top_zip_path = output_genion_top_zip,
       output_tpr_path    = output_gppmd_tpr,
       input_cpt_path     = output_npt_cpt,
       properties         = prop)

2022-04-22 10:39:01,942 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-22 10:39:01,947 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_genion_top.zip
2022-04-22 10:39:01,948 [MainThread  ] [INFO ]  to:
2022-04-22 10:39:01,949 [MainThread  ] [INFO ]  ['e407a8e2-2a38-4bca-867b-83869a553344/ligand.top', 'e407a8e2-2a38-4bca-867b-83869a553344/ligand_params.itp', 'e407a8e2-2a38-4bca-867b-83869a553344/ligand_posres.itp', 'e407a8e2-2a38-4bca-867b-83869a553344/posre.itp']
2022-04-22 10:39:01,950 [MainThread  ] [INFO ]  Unzipping: 
2022-04-22 10:39:01,950 [MainThread  ] [INFO ]  protein_ligand_genion_top.zip
2022-04-22 10:39:01,951 [MainThread  ] [INFO ]  To: 
2022-04-22 10:39:01,951 [MainThread  ] [INFO ]  e407a8e2-2a38-4bca-867b-83869a553344/ligand.top
2022-04-22 10:39:01,952 [MainThread  ] [INFO ]  e407a8e2-2a38-4bca-867b-83869a553344/ligand_params.itp
2022-04-22 10:39:01,953 [MainThread  ] [INFO ]  e407a8e2-2a38-4bca-86

0

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [42]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = pdbName+'_'+ligandName+'_md.trr'
output_md_gro = pdbName+'_'+ligandName+'_md.gro'
output_md_edr = pdbName+'_'+ligandName+'_md.edr'
output_md_log = pdbName+'_'+ligandName+'_md.log'
output_md_cpt = pdbName+'_'+ligandName+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr,
      output_trr_path=output_md_trr,
      output_gro_path=output_md_gro,
      output_edr_path=output_md_edr,
      output_log_path=output_md_log,
      output_cpt_path=output_md_cpt)

2022-04-22 10:39:02,262 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-22 10:39:02,263 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:35,293 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppmd.tpr -o protein_ligand_md.trr -c protein_ligand_md.gro -e protein_ligand_md.edr -g protein_ligand_md.log -cpo protein_ligand_md.cpt

2022-04-22 10:47:35,294 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:35,294 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/simulation/md-biobb
Command line:
  gmx -nobackup -nocopyright mdrun -s protein_ligand_gppmd.tpr -o protein_ligand_md.trr -c protein_ligand_md.gro -e protein_ligand_md.edr -g protein_ligand_md.log -cpo protein_ligand_md.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see l

0

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [43]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbName+'_'+ligandName+'_rms_first.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

2022-04-22 10:47:35,310 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:35,396 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_gppmd.tpr -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_md.trr -o protein_ligand_rms_first.xvg -xvg none

2022-04-22 10:47:35,397 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:35,398 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-22 10:47:35,399 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph J

0

In [44]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbName+'_'+ligandName+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms (input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
         properties=prop)

2022-04-22 10:47:35,419 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:35,507 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_gppmin.tpr -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_md.trr -o protein_ligand_rms_exp.xvg -xvg none

2022-04-22 10:47:35,508 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:35,509 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-22 10:47:35,509 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Ju

0

In [45]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig4.png' width="50%" height="50%"></img>

In [46]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbName+'_'+ligandName+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
         properties=prop)

2022-04-22 10:47:35,579 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:35,643 [MainThread  ] [INFO ]  echo "Backbone" | gmx gyrate -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_gppmin.tpr -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_md.trr -o protein_ligand_rgyr.xvg -xvg none

2022-04-22 10:47:35,644 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:35,645 [MainThread  ] [INFO ]  Selected 4: 'Backbone'

2022-04-22 10:47:35,646 [MainThread  ] [INFO ]                        :-) GROMACS - gmx gyrate, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimi

0

In [47]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig5.png' width="50%" height="50%"></img>

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein-ligand complex system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image**:
     * Wrapper of the GROMACS trjconv module for correcting periodicity (image) from a given GROMACS compatible trajectory file.
     
     
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**:
     * Wrapper of the GROMACS trjconv module for converting between GROMACS compatible structure file formats and/or extracting a selection of atoms.
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [48]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbName+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein_Other',
    'output_selection': 'Protein_Other',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image (input_traj_path  = output_md_trr,
           input_top_path   = output_gppmd_tpr,
           input_index_path = output_complex_ndx,
           output_traj_path = output_imaged_traj, 
           properties       = prop)

2022-04-22 10:47:35,737 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:35,837 [MainThread  ] [INFO ]  echo "Protein_Other" "Protein_Other" | gmx trjconv -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_md.trr -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_gppmd.tpr -fit none -o protein_imaged_traj.trr -n /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_index.ndx -center -pbc mol -ur compact

2022-04-22 10:47:35,838 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:35,838 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for centering
Selected 24: 'Protein_Other'
Select group for output
Selected 24: 'Protein_Other'

2022-04-22 10:47:35,839 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. 

0

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [49]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbName+'_md_dry.gro'
prop = {
    'selection':  'Protein_Other'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path = output_md_gro,
                input_top_path       = output_gppmd_tpr,
                input_index_path     = output_complex_ndx,
                output_str_path      = output_dry_gro, 
                properties           = prop)

2022-04-22 10:47:35,871 [MainThread  ] [INFO ]  Not using any container
2022-04-22 10:47:36,024 [MainThread  ] [INFO ]  echo "Protein_Other" | gmx trjconv -f /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_md.gro -s /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_gppmd.tpr -o protein_md_dry.gro -n /home/lg/repos/javerianaMD/simulation/md-biobb/protein_ligand_index.ndx

2022-04-22 10:47:36,025 [MainThread  ] [INFO ]  Exit code 0

2022-04-22 10:47:36,026 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 24: 'Protein_Other'

2022-04-22 10:47:36,027 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van

0

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [50]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

NGLWidget(max_frame=5)

<img src='images/trajectory.gif'></img>

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 